In [1]:
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et
import pandas as pd
import urllib.request
from shapely.geometry import Point, shape
import json
import math
import networkx as nx

In [2]:
def get_haversine_distance(point_1, point_2):
    """
    Calculate the distance between any 2 points on earth given as [lon, lat]
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [point_1[0], point_1[1], 
                                                point_2[0], point_2[1]])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371000 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [3]:
city='Detroit'
SIM_AREA_PATH='./cities/'+city+'/clean/table_area.geojson'
FLOYD_PREDECESSOR_PATH='./cities/'+city+'/clean/fw_result.json'
INT_NET_DF_FLOYD_PATH='./cities/'+city+'/clean/sim_net_df_floyd.csv'
INT_NET_COORDINATES_PATH='./cities/'+city+'/clean/sim_net_node_coords.json'

sim_area=json.load(open(SIM_AREA_PATH))

full_area=[shape(f['geometry']) for f in sim_area['features']]
bounds=[shp.bounds for shp in full_area]
boundsAll=[min([b[0] for b in bounds]), #W
               min([b[1] for b in bounds]), #S
               max([b[2] for b in bounds]), #E
               max([b[3] for b in bounds])] #N

In [4]:
query='http://api.openstreetmap.org/api/0.6/map?bbox='+','.join([str(b) for b in boundsAll])
file = urllib.request.urlopen(query)
xData=et.parse(file)
file.close()

root=xData.getroot()
root.tag

nodeLat=[]
nodeLong=[]
nodeId=[]
aNodes=[]
bNodes=[]
hwTags=[]
names=[]
oneway=[]
ref=[]
maxspeed=[]
osmid=[]

for child in root:
    if child.tag=='node':
        nodeLat.extend([child.get('lat')])
        nodeLong.extend([child.get('lon')])
        nodeId.extend([child.get('id')])
    elif child.tag=='way':
        highwayTag, name, oneW, re, mSp=[], [], [], [], []
        for t in child.findall('tag'):
            key=t.get('k')
            if key=='highway':
                value=t.get('v')
                highwayTag.extend([value])
        #now confirmed if the child is a way: if so, loop through the tags again and save them
        if any(highwayTag):
            for t in child.findall('tag'):
                if t.get('k')=='name':
                    name=t.get('v')
                if t.get('k')=='oneway':
                    oneW=t.get('v')
                if t.get('k')=='ref':
                    re=t.get('v')
                if t.get('k')=='maxspeed':
                    mSp=t.get('v')
            first=1
            for gChild in child:
                if gChild.tag=='nd':
                    if first==1:
                        #aNodes.extend([gChild.get('ref')])
                        lastNode=gChild.get('ref')
                    else:
                        aNodes.extend([lastNode])
                        lastNode=gChild.get('ref')
                        bNodes.extend([lastNode])
                        hwTags.extend([highwayTag[0]])
                        names.extend([name])
                        oneway.extend([oneW])
                        ref.extend([re])
                        maxspeed.extend([mSp])
                        osmid.extend([list(child.getiterator())[0].attrib['id']])
                    first=0

In [5]:
latLong= [[nodeLat[i], nodeLong[i]] for i in range(len(nodeId))]
LatLongDict = dict(zip(nodeId, latLong))

aNodeLat=[LatLongDict[aNodes[i]][0] for i in range(len(aNodes))]
aNodeLon=[LatLongDict[aNodes[i]][1] for i in range(len(aNodes))]
bNodeLat=[LatLongDict[bNodes[i]][0] for i in range(len(bNodes))]
bNodeLon=[LatLongDict[bNodes[i]][1] for i in range(len(bNodes))]

network=pd.DataFrame({'aNodes':aNodes, 'bNodes':bNodes, 
'aNodeLat':aNodeLat, 'aNodeLon':aNodeLon,'bNodeLat':bNodeLat,'bNodeLon':bNodeLon, 'type':hwTags, 
'name':names, 'speed':maxspeed, 'ref':ref, 'osmid':osmid, 'oneway':oneway})

Add link distances

In [6]:
network['distance']=network.apply(lambda row: get_haversine_distance([float(row['aNodeLon']), float(row['aNodeLat'])],
                                                                     [float(row['bNodeLon']), float(row['bNodeLat'])]), axis=1)

Add other-way nodes

In [7]:
copy_edges=network.copy()
for i in range(len(copy_edges)):
    if copy_edges.iloc[i]['oneway']!='yes':
        temp_link=copy_edges.iloc[i].copy()
        temp_from_node = temp_link['aNodes']
        temp_link['aNodes']=temp_link['bNodes']
        temp_link['bNodes']=temp_from_node
        network.loc[len(network)]=temp_link.copy()

Index by to and from nodes and remove duplicates

In [8]:
network['aNode_bNode']=network.apply(lambda row: '{}_{}'. format(row['aNodes'], row['bNodes']), axis=1)
network=network.set_index('aNode_bNode')
len(network)

23065

In [9]:
network = network.loc[~network.index.duplicated(keep='first')]
len(network)

23055

In [10]:
%matplotlib inline

In [11]:
from ipyleaflet import Map, Polyline, basemaps, basemap_to_tiles, Circle, Marker

In [12]:
polylines=[]
m = Map(center = (aNodeLat[0], aNodeLon[0]), zoom =14, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
lines=[]
for ind, row in network.iterrows():
    lines.append([[float(row['aNodeLat']), float(row['aNodeLon'])], [float(row['bNodeLat']), float(row['bNodeLon'])]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='white',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [13]:
network.head()

,aNodes,bNodes,aNodeLat,aNodeLon,bNodeLat,bNodeLon,type,name,speed,ref,osmid,oneway,distance
aNode_bNode,,,,,,,,,,,,,
62547367_62547369,62547367,62547369,42.3386563,-83.0559908,42.3382589,-83.0568615,motorway_link,[],[],[],8727492,yes,84.108654
62547369_2149010904,62547369,2149010904,42.3382589,-83.0568615,42.3381456,-83.0570798,motorway_link,[],[],[],8727492,yes,21.924036
2149010904_62547371,2149010904,62547371,42.3381456,-83.0570798,42.3380308,-83.0572761,motorway_link,[],[],[],8727492,yes,20.573631
62547371_4216073828,62547371,4216073828,42.3380308,-83.0572761,42.3378798,-83.0575108,motorway_link,[],[],[],8727492,yes,25.574507
62547437_62547439,62547437,62547439,42.3326578,-83.0620911,42.3330202,-83.0622765,motorway_link,[],[],[],8727497,yes,43.082564


# Find subset of links where at least one node is in the simulation area

In [14]:
print(len(LatLongDict))
sim_area_nodes=set()
# check if each node in any sim area, if so add to list
for node_num in LatLongDict:
    for feat in sim_area['features']:
        if shape(feat['geometry']).contains(Point(
            float(LatLongDict[node_num][1]), 
            float(LatLongDict[node_num][0]))):
            sim_area_nodes.add(node_num)
len(sim_area_nodes)

27063


15333

In [15]:
print(len(network))
network=network.loc[((network['aNodes'].isin(sim_area_nodes))|
    (network['bNodes'].isin(sim_area_nodes)))]
print(len(network))

23055
13784


# Plot network inside sim area only

In [16]:
polylines=[]
m = Map(center = (aNodeLat[0], aNodeLon[0]), zoom =15, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
lines=[]
for ind, row in network.iterrows():
    lines.append([[float(row['aNodeLat']), float(row['aNodeLon'])], [float(row['bNodeLat']), float(row['bNodeLon'])]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='white',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [17]:
edges_types=set(network['type'])
edges_types

{'footway',
 'motorway',
 'motorway_link',
 'path',
 'pedestrian',
 'primary',
 'primary_link',
 'residential',
 'secondary',
 'secondary_link',
 'service',
 'tertiary',
 'track',
 'unclassified'}

In [18]:
drive_types=[t for t in edges_types if t not in ['footway', 'path', 'steps', 'pedestrian', 'track']]

In [19]:
drive_edges=network.loc[network['type'].isin(drive_types)]

In [20]:
len(drive_edges)

5946

In [21]:
polylines=[]
m = Map(center = (aNodeLat[0], aNodeLon[0]), zoom =15, layers=[basemap_to_tiles(basemaps.CartoDB.DarkMatter)])
lines=[]
for ind, row in drive_edges.iterrows():
    lines.append([[float(row['aNodeLat']), float(row['aNodeLon'])], [float(row['bNodeLat']), float(row['bNodeLon'])]])

lineLayer = Polyline(
    locations = lines,
    weight=1,
    opacity=1,
    color='white',
    fill=False
)
m.add_layer(lineLayer)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

Create Networkx DiGraph

In [22]:
G=nx.DiGraph()
for i, row in drive_edges.iterrows():
    G.add_edge(row['aNodes'], row['bNodes'], 
               weight=row['distance'])

Solve shortest paths using Floyd Warshall

In [23]:
fw_result=nx.floyd_warshall_predecessor_and_distance(G, weight='weight')

In [24]:
# Create mapping from nodes to link attributes to speed up queries
nodes_to_link_attributes={}
for ind, row in drive_edges.iterrows():
    nodes_to_link_attributes['{}_{}'.format(row['aNodes'], row['bNodes'])]={
        'distance': row['distance'],
        'from_coord': [float(row['aNodeLon']), float(row['aNodeLat'])],
        'to_coord': [float(row['bNodeLon']), float(row['bNodeLat'])]}

In [25]:
def get_node_path_from_fw(all_pred, from_node, to_node):
    if from_node==to_node:
        return []
    pred=to_node
    path=[pred]
    while not pred==from_node:
        pred=all_pred[from_node][pred]
        path.insert(0,pred)
    return path
        
def get_path_coords_distances(drive_edges, path):
    coords, distances =[], []
    for node_ind in range(len(path)-1):
        from_node=path[node_ind]
        to_node=path[node_ind+1]
        link_attributes=nodes_to_link_attributes['{}_{}'.format(from_node, to_node)]
        distances+=[link_attributes['distance']]
        coords+=[link_attributes['from_coord']]
    coords+= [link_attributes['to_coord']]
    # add the final coordinate of the very last segment
    return coords, distances

# Test The Results

In [26]:
from_node, to_node=drive_edges.iloc[0]['aNodes'], drive_edges.iloc[10]['aNodes']

In [27]:
# using dijkstras
print('Shortest path by num links: {}'.format(nx.shortest_path(G, from_node, to_node)))
print('Shortest path by distance: {}'.format(nx.dijkstra_path(G, from_node, to_node, weight='weight')))
print('Distance of Shortest path: {}'.format(nx.dijkstra_path_length(G, from_node, to_node, weight='weight')))

Shortest path by num links: ['62574668', '62574679', '62574682', '62574684', '62574687', '62597342', '62549168', '62549171', '62549173', '62549175', '62549177']
Shortest path by distance: ['62574668', '62574679', '62574682', '62574684', '62574687', '62597342', '62549168', '62549171', '62549173', '62549175', '62549177']
Distance of Shortest path: 253.45259979991894


In [28]:
path=get_node_path_from_fw(fw_result[0], from_node, to_node)
coords, distances=get_path_coords_distances(drive_edges, path)
print(path)
print(coords)
print(sum(distances))

['62574668', '62574679', '62574682', '62574684', '62574687', '62597342', '62549168', '62549171', '62549173', '62549175', '62549177']
[[-83.0727618, 42.3332176], [-83.0721618, 42.3333486], [-83.0720108, 42.3333736], [-83.0717868, 42.3333956], [-83.0716488, 42.3334096], [-83.0707488, 42.3334686], [-83.0706948, 42.3334756], [-83.0704528, 42.3335206], [-83.0701628, 42.3335886], [-83.0699368, 42.3336556], [-83.0697718, 42.3337126]]
253.45259979991894


In [29]:
import datetime
count=0
then=datetime.datetime.now()
for from_node in fw_result[0]:
    print(len(fw_result[0][from_node]))
    for to_node in fw_result[0][from_node]:
        path=get_node_path_from_fw(fw_result[0], from_node, to_node)
        coords, distances=get_path_coords_distances(drive_edges, path)
        count+=1
now=datetime.datetime.now()
interval=(now-then).seconds
print('{} seconds per query'.format(interval/count))

43
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
2697
58
57
56
55
54
53
52
51
50
2760
2759
2758
2757
2789
2757
2756
2755
2754
2753
2752
2751
2750
2749
2748
2747
2746
2745
2744
2743
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2777
2713
2712
2711
2710
2709
10
4
3
2
1
2730
2729
2728
2727
2726
2725
2702
2701
2700
2699
2698
2697
57
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2697
12
11
10
9
2703
2702
2701
40
23
22
21
20
19
18
21
20
19
18
17
16
15
2740
54
53
52
51
50
49
48
47
46
45
44
2697
2697
2697
2697
2697
2697
2697
2698
2697
2697
2697
2701
2700
2699
2698
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
269

2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697
2697


In [30]:
# Create a coordinate lookup for all the nodes included in the final graph

In [31]:
all_nodes=set(list(drive_edges['aNodes'])+list(drive_edges['bNodes']))
node_to_coordinates={}
for node in LatLongDict:
    if node in all_nodes:
        node_to_coordinates[node]=[float(LatLongDict[node][1]), float(LatLongDict[node][0])]

In [32]:
json.dump(fw_result[0], open(FLOYD_PREDECESSOR_PATH, 'w'))
drive_edges.to_csv(INT_NET_DF_FLOYD_PATH)
json.dump(node_to_coordinates, open(INT_NET_COORDINATES_PATH, 'w'))